
# QCI
#### Author: Jan T. (IR) en Team D&A

In [1]:
###Load packages
import pandas       as pd   #required for data wrangling
import pingouin     as pg   #required for Cronbach's Alpha
import numpy        as np   #required for NaN

In [4]:
### Load data
qcidata = pd.read_csv("QCIdata_volledig_anoniem.csv", sep=";")

## Data cleaning
Remove all values 0, 6, 7 and 8 (corresponding to some missing data type), since they will influence the outcome. In this case, the best method is to replace these values by 'NaN' (not a number). 

In [5]:
missing_numbers = [0,6,7,8]
qcidata_clean = qcidata.replace(missing_numbers[0], np.NaN)

## Interne consistentie
Om interne consistentie per subschaal te meten is [Cronbach's Alpha](https://www.scribbr.nl/statistiek/cronbachs-alpha/) gebruikt, deze is geïnterpreteerd met gebruik van de volgende [bron](https://towardsdatascience.com/cronbachs-alpha-theory-and-application-in-python-d2915dd63586).

### Definieer subschalen
Elke subschaal bestaat uit een aantal vragen. Hier wordt voor elke subschaal gedefinieerd uit welke vragen deze bestaat, en welke subschalen er totaal zijn. 

In [6]:
# Definieer vragen per subschalen
doeltreffendheid = ['Vraag3', 'Vraag4', 'Vraag5', 'Vraag6']
affectiviteit = ['Vraag7', 'Vraag8', 'Vraag9']

# Alle subschalen verzameld
subschalen = [doeltreffendheid, affectiviteit]
N_subschalen = len(subschalen)

In [7]:
alphas = []
for i in range(0,N_subschalen):
    alpha = (pg.cronbach_alpha(data=qcidata[subschalen[i]]))[0]
    alphas.append(alpha)
alphas

[0.7650986344675027, 0.8043022019102172]

In [ ]:
alpha_threshold = 0.7
test